In [1]:
import sys
from bs4 import BeautifulSoup
import requests
import datetime as dt
from PIL import Image
import helpers as hf

import yaml
from unidecode import unidecode
import pandas as pd
from PIL import Image, ImageOps, ImageDraw, ImageFont
import json
import numpy as np
from instagrapi import Client as instaClient
import tempfile
from PIL import Image
from io import BytesIO
with open("info/design_config.yml", "r") as f:
    design_config = yaml.safe_load(f)

design_config,shop_config = hf.get_config('cbb')
cfbd_loader = hf.cfbp_handler(design_config['cfbd_api'])

private file size: 2640


In [15]:
teams = cfbd_loader.get_team_info()

In [116]:
# check if printify ptoduct is posted
headers_printify = {
    "Authorization": f"Bearer {shop_config['printify_access']}",
    "Content-Type": "application/json",
}
headers_shopify = {
            "X-Shopify-Access-Token": shop_config["shopify_access"],
            "Content-Type": "application/json",
        }

In [117]:
products_link = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2024-01/products.json?limit=250"
response = requests.get(products_link, headers=headers_shopify)
response.headers["X-Shopify-Shop-Api-Call-Limit"]

'1/40'

In [18]:
ii_response = requests.get('https://cdn.shopify.com/s/files/1/0685/3509/1421/files/2333403980485870616_2048.jpg?v=1708887700')
img = Image.open(BytesIO(ii_response.content))

In [23]:
ii_response.headers['X-Rate-Limit-Remaining']

KeyError: 'x-rate-limit-remaining'

In [13]:
ii_response_test = {
     "id": "5d39b411749d0a000f30e0f4",
     "title": "Product",
     "description": "Good product",
     "tags": [
         "Home & Living",
         "Stickers"
     ],
     "options": [
         {
             "name": "Size",
             "type": "size",
             "values": [
                 {
                     "id": 2017,
                     "title": "2x2\""
                 },
                 {
                     "id": 2018,
                     "title": "3x3\""
                 },
                 {
                     "id": 2019,
                     "title": "4x4\""
                 },
                 {
                     "id": 2020,
                     "title": "6x6\""
                 }
             ]
         },
         {
             "name": "Type",
             "type": "surface",
             "values": [
                 {
                     "id": 2114,
                     "title": "White"
                 }
             ]
         }
     ],
     "variants": [
         {
             "id": 45740,
             "sku": "866375988",
             "cost": 134,
             "price": 400,
             "title": "2x2\" / White",
             "grams": 10,
             "is_enabled": True,
             "is_default": True,
             "is_available": True,
             "is_printify_express_eligible": True,
             "options": [
                 2017,
                 2114
             ]
         },
         {
             "id": 45742,
             "sku": "866375989",
             "cost": 149,
             "price": 400,
             "title": "3x3\" / White",
             "grams": 10,
             "is_enabled": True,
             "is_default": False,
             "is_available": True,
             "is_printify_express_eligible": True,
             "options": [
                 2018,
                 2114
             ]
         },
         {
             "id": 45744,
             "sku": "866375990",
             "cost": 187,
             "price": 400,
             "title": "4x4\" / White",
             "grams": 10,
             "is_enabled": True,
             "is_default": False,
             "is_available": True,
             "is_printify_express_eligible": True,
             "options": [
                 2019,
                 2114
             ]
         },
         {
             "id": 45746,
             "sku": "866375991",
             "cost": 216,
             "price": 400,
             "title": "6x6\" / White",
             "grams": 10,
             "is_enabled": True,
             "is_default": False,
             "is_available": True,
             "is_printify_express_eligible": True,
             "options": [
                 2020,
                 2114
             ]
         }
     ],
     "images": [
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2187/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": True
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2188/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2189/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2190/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2191/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2192/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2193/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2194/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2195/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2196/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2197/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2198/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2199/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2200/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2201/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2202/product.jpg",
             "variant_ids": [
                 45740
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2187/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2188/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2189/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": True
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2190/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2191/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2192/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2193/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2194/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2195/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2196/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2197/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2198/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2199/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2200/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2201/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45742/2202/product.jpg",
             "variant_ids": [
                 45742
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2187/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2188/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2189/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2190/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": True
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2191/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2192/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2193/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2194/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2195/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2196/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2197/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2198/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2199/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2200/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2201/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45744/2202/product.jpg",
             "variant_ids": [
                 45744
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2187/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2188/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2189/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2190/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2191/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": True
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2192/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2193/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2194/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2195/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2196/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2197/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2198/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2199/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2200/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2201/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         },
         {
             "src": "https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45746/2202/product.jpg",
             "variant_ids": [
                 45746
             ],
             "position": "front",
             "is_default": False
         }
     ],
     "created_at": "2019-07-25 13:52:17+00:00",
     "updated_at": "2019-07-25 13:52:18+00:00",
     "visible": True,
     "is_locked": False,
     "is_printify_express_eligible": True,
     "is_printify_express_enabled": True,
     "blueprint_id": 384,
     "user_id": 1337,
     "shop_id": 1337,
     "print_provider_id": 1,
     "print_areas": [
         {
             "variant_ids": [
                 45740,
                 45742,
                 45744,
                 45746
             ],
             "placeholders": [
                 {
                     "position": "front",
                     "images": [
                         {
                             "id": "5d15ca551163cde90d7b2203",
                             "name": "Asset 65@3x.png",
                             "type": "image/png",
                             "height": 1200,
                             "width": 1200,
                             "x": 0.5,
                             "y": 0.5,
                             "scale": 1,
                             "angle": 0
                         }
                     ]
                 }
             ],
             "background": "#ffffff"
         }
     ],
     "sales_channel_properties": []   
  }
  

In [16]:
ii_response_test["images"][2]['src']

'https://images.printify.com/mockup/5d39b411749d0a000f30e0f4/45740/2189/product.jpg'

In [17]:
a = [1,23,3]
b = [0,0,0]

for c,d in zip(a,b):
    print(c,d)

1 0
23 0
3 0


In [8]:
insta_image = json.loads(ii_response_test)["images"][2]
ii_response = requests.get(insta_image)
insta_image = Image.open(BytesIO(ii_response.content))

JSONDecodeError: Expecting ',' delimiter: line 16 column 36 (char 370)

In [5]:
instagram = hf.instagrammer(
    un=shop_config["instagram"]["username"],
    pw=shop_config["instagram"]["password"],
    email=shop_config["instagram"]["eusername"],
    emailpw=shop_config["instagram"]["epassword"],
    
)

In [6]:
instagram.single_post(img,'test')

Attempting single image post


In [15]:
call_limit = response.headers["X-Shopify-Shop-Api-Call-Limit"]
call_split = [int(x) for x in call_limit.split('/')]
remaining_calls = call_split[1]-call_split[0]

In [10]:
print(len(prods))
print(len(prods2))

472
236


In [ ]:
import time

In [4]:
graphql_url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2023-04/graphql.json"


In [5]:
big_body ="""
            {
            product(id:"gid://shopify/Product/prod_id") {
                title
                media(first:5) {
                edges {
                    node {
                    ... fieldsForMediaTypes
                    }
                }
                }
            }
            }

            fragment fieldsForMediaTypes on Media {
            alt
            mediaContentType
            preview {
                image {
                id
                altText
                url
                }
            }
            status
            ... on Video {
                id
                sources {
                format
                height
                mimeType
                url
                width
                }
                originalSource {
                format
                height
                mimeType
                url
                width
                }
            }
            ... on ExternalVideo {
                id
                host
                embeddedUrl
            }
            ... on Model3d {
                sources {
                format
                mimeType
                url
                }
                originalSource {
                format
                mimeType
                url
                }
            }
            ... on MediaImage {
                id
                image {
                altText
                url
                }
            }
            }
            """

In [6]:
prod_id = 8368514007261

In [7]:
all_p = ify_user.recur_get_products()

In [9]:
for x in all_p:
    if x['id'] == prod_id:
        prod = x

In [12]:
prod_gql = prod["admin_graphql_api_id"]
body = big_body.replace("prod_id", str(prod_id))
# now that I have the images I can reorder them
url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2024-01/graphql.json"
# from this response i can get my images
response = requests.post(
    url=url, headers=headers_shopify, json={"query": body}
)

images = json.loads(response.text)["data"]["product"]["media"]["edges"]
image0 = images[0]["node"]["id"]
image2 = images[2]["node"]["id"]

In [14]:
title = prod['title']

In [15]:
alt_text_query = "mutation productUpdateMedia($media: [UpdateMediaInput!]!, $productId: ID!) { productUpdateMedia(media: $media, productId: $productId) { media { alt } } }"

alt_text_var = {
"media": [
    {
    "alt": title+' back view',
    "id": image0
    }
],
"productId": prod_gql
}

alt_resp = requests.post(
    graphql_url, headers=headers_shopify, json={"query": alt_text_query, "variables": alt_text_var}
)

In [ ]:
import numpy as np

In [16]:
'currentlyAvailable' in str(alt_resp.headers)

False

In [26]:
json.loads(alt_resp.text)['extensions']['cost']['throttleStatus']['currentlyAvailable'] < 100:

False

In [32]:
alt_resp.headers["extensions"]

KeyError: 'extensions'